# Reader Monad

## Outline

* Incentive for the Reader monad

* The Reader monad
  - Definition of the Reader monad

  - Helper functions

  - MonadReader type class

  - Simple example
  
* Reader monad examples

In this lesson, we will learn about the Reader monad type and how you can use it. First we present the reason why introducing the Reader monad. Then we show the Reader monad definition and helper functions. We also learn a new ways how to create a type class by looking at the `MonadReader` type class. In the end we show some code examples that use the Reader monad.

## Incentive for Reader monad

Imagine we have a global environment which is represented by a variable that holds multiple data as e.g. a record syntax variable. 

If we have several functions that need to use part of that data and they call each other, we need to pass that variable from one to another.

Below is an example where we read 4 variables from the operating system environment and store them into a variable of type `Environment`. 

We process this data in 4 function calls and the last function writes the processed data to a configuration file.

In [ ]:
import Data.Maybe (fromMaybe)
import System.Environment (lookupEnv)

data Environment = Environment
    { param1 :: String
    , param2 :: String
    , param3 :: String
    , param4 :: String
    }

loadEnv :: IO Environment
loadEnv = do
  p1 <- lookupEnv "PATH"
  p2 <- lookupEnv "SHELL"
  p3 <- lookupEnv "HOSTNAME"
  p4 <- lookupEnv "HOME"
  return $ Environment
    (fromMaybe "" p1)
    (fromMaybe "" p2)
    (fromMaybe "" p3)
    (fromMaybe "" p4)

func1 :: Environment -> (String, FilePath)
func1 env = func2 env content
    where content = "System path includes: \n" ++ param1 env ++ "\n\n"

func2 :: Environment -> String -> (String, FilePath)
func2 env content = func3 env updatedContent
    where updatedContent = content ++ "Shell used is: \n" ++ param2 env ++ "\n\n"

func3 :: Environment -> String -> (String, FilePath)
func3 env content = func4 env updatedContent
    where updatedContent = content ++ "Hostname is: \n" ++ param3 env ++ "\n\n"

func4 :: Environment -> String -> (String, FilePath)
func4 env content = (updatedContent, filePath)
    where updatedContent = content ++ "Current home directory is: \n" ++ param4 env ++ "\n\n"
          filePath = param4 env ++ "/conf.txt"

We can now write the main execution of our program where we read in the environment and process the data with our 4 functions.

In [ ]:
main :: IO ()
main = do
  env <- loadEnv
  let (updatedContent, filePath) = func1 env

  print "Writing configuration to file: conf.txt"
  writeFile filePath updatedContent

main

This is a simple example, but as complexity of programs increases you have to pass around a lot of data. 

For this reason the **Reader monad** was created which can store data in its environment variable.  

The data in the environment variable can then be passed between different Reader monads in the background.

## The Reader monad

### Definition of the Reader monad

The Haskell definition of the `Reader` type is defined in terms of the `ReaderT` monad transformer:
```haskell
type Reader r = ReaderT r Identity
```

It is contained in the **Control.Monad.Trans.Reader** module that comes with the **transformers** package. 

We will learn about Monad transformers in lesson 25.

You can also define the Reader monad without using a monad transformer that works equally as the one above. 

We will chose this aproach in this lesson. Let's first look at the definition of the **Reader** type:
```haskell
newtype Reader e a = Reader { runReader :: e -> a }
```

We see that the `Reader` data constructor holds a function that can be accessed with the name `runReader`.

It takes in a environment variable of type `e` and returns a variable of type `a`.

We can now create a Monad instance for `Reader e` and not just `Reader`.

This means the type of our environment variable will remain the same after we apply the `>>=` operator.
```haskell
(>>=) :: Reader e a        ->
         (a -> Reader e b) ->
         Reader e b
```

The monad instance for `Reader e` is then defined as:
```haskell
instance Monad (Reader e) where 
  return a = Reader $ const a
  readerMonad >>= f = Reader $ \env -> 
    let a = runReader readerMonad env
    in runReader (f a) env
```

We also need to define the Functor and Applicative instances.
```haskell
instance Functor (Reader e) where
fmap f readerFunctor = Reader $ \env ->
    runReader readerFunctor (f env)

instance Applicative (Reader e) where
pure a = Reader $ const a
f <*> g = Reader $ \env ->
    let h = runReader f env
        readerApplicative = runReader g (h env)
    in runReader readerApplicative env
```

### Helper functions

The **Control.Monad.Trans.Reader** module defines also some helper functions that work with the Reader moand. 

The `ask` function retrieves the value from the enviroment variable.
```haskell
ask :: Reader e e 
ask = Reader id
```

It is used inside a Reader Monad. To get the environment variable you also need to use the `<-` oprator that takes variables out of the monadic context.
```haskell
env <- ask
```

The `local` function takes in a function and a Reader moand and updates the Reader monad by applying the function to the environment variable.
```haskell
local :: (e -> e) -> Reader e a -> Reader e a
local f readerM = Reader $ runReader readerM . f
```

It can also be used inside inside a Reader monad. Same as for `ask` function if you want to get the modified enviroment variable you need to use the `<-` operator. 

Another helper function is the `asks` function which performs the inverse operation of the `runReader` function.

It takes in a function of type `e -> a` and produces a Reader monad.
```haskell
asks :: (e -> a) -> Reader e a
asks f = do
    r <- ask
    return (f e)
```

The `asks` function is used inside a Reader to update the environment variable by applying the input function to it. 

### MonadReader type class

The Reader monad and the helper functions presented in the previous two chapters can be also found in the **Control.Monad.Reader** module.

This module is part of the **mtl** package that is a collection of monad classes, extending the **transformers** package.

The module defines the `MonadReader` type class that contains the functions `ask`, `local` and `reader`.

They are equivalent to the functions describer in the previous chapter.

```haskell
class Monad m => MonadReader r m | m -> r where
    ask   :: m r
    ask = reader id

    local :: (r -> r) -> m a -> m a

    reader :: (r -> a) -> m a
    reader f = do
      r <- ask
      return (f r)
    {-# MINIMAL (ask | reader), local #-}
```

The minimal complete definition is the `local` function and then either the `ask` or the `reader` function.

To be able to declare a type class parameterized with two types you need to use the *MultiParamTypeClasses* language pragma. 

If the type `m` which in reality for us will be `(Reader r)`, is defiinng the type `r` we can state this dependency with the `| m -> r` statement. 

To be able to use such dependency statements in type class declarations we need to include the *FunctionalDependencies* language pragma. 

For the `Reader e` monad the instance of the MonadReader type class would be:
```haskell
instance MonadReader e (Reader e) where
    ask :: Reader e e
    ask = Reader id
    
    local :: (e -> e) -> Reader e a -> Reader e a
    local f readerM = Reader $ runReader readerM . f
```

To be able to write such an instance we would also need to include the language pragmas *FlexibleInstances* and *InstanceSigs*.

The first let's you work with concrete types when creating an type class instances as e.g. if you would want to create an instance for `Maybe Int` instead of `Maybe a`.

The second enables to also write out type signatures as part of typeclass instance declarations.

### Simple example

Below is a simple Reader monad example where we use the `ask` and `runReader` functions.

The programm asks the user for its age and then prints our his age and double times his age.

In [ ]:
import Control.Monad.Reader

age :: Reader Int String
age = do
    env <- ask
    return ("Your age is: " ++ (show env))

ageDouble :: Reader Int String
ageDouble = do
    env <- ask
    return ("Two times your age is: " ++ (show $ 2*env))

bothAges :: Reader Int String
bothAges = do
    a <- age
    ad <- ageDouble
    return (a ++ "\n" ++ ad)

main :: IO ()
main = do
    putStrLn "Input your age:"
    myAge <- read <$> getLine
    putStrLn $ (runReader bothAges) myAge

main

We see that when we call the `age` and `ageDouble` Reader monads inside the `bothAges` Reader monad, the environment variabe of the `bothAges` Reader monad is passed onto them.

## Reader monad examples

In our first example we update the code from the previous chapter such that we use the `local` function.

We modily the environment variable of the `age` and `ageDouble` Reader monads by applying the function `add2Years` that icreases the age for 2.

In [ ]:
age :: Reader Int String
age = do
    env <- ask
    return ("In two years your age will be: " ++ (show env))

ageDouble :: Reader Int String
ageDouble = do
    env <- ask
    return ("Two times your age in two years will be: " ++ (show $ 2*env))

bothAges :: Reader Int String
bothAges = do
    let age' = local add2Years age
        ageDouble' = local add2Years ageDouble
    a <- age'
    ad <- ageDouble'
    return (a ++ "\n" ++ ad)

add2Years :: Int -> Int
add2Years years = years + 2

main :: IO ()
main = do
    putStrLn "Input your age:"
    myAge <- read <$> getLine
    putStrLn $ (runReader bothAges) myAge

main

We see that the environment variable from the `bothAges` Reader monad gets passed onto the other two Reader monads after the function `add2Years` is applied to it.

Below is another example on how to lookup variables if you have them stored in a list (taken and adapted from HaskellWiki [(1)](https://wiki.haskell.org/All_About_Monads#The_Reader_monad)).

The list contains tuples where the first part is the variable name and the second is variable either of type `String` or of type `Template`. 

What we want to show here is that you can also define a function that takes in data and returns a reader monad. We use the `asks` function.

In [ ]:
data Template = S String | V Template | T Template deriving Show

data Environment = Env {templates::[(String,Template)],
                        variables::[(String,String)]}

-- lookup a variable from the environment
lookupVar :: String -> Environment -> Maybe String
lookupVar name env = lookup name (variables env)

-- lookup a template from the environment
lookupTemplate :: String -> Environment -> Maybe Template
lookupTemplate name env = lookup name (templates env)

-- resolve a template into a string
resolve :: Template -> Reader Environment (String)
resolve (S s)    = return s
resolve (V t)    = do varName  <- resolve t
                      varValue <- asks (lookupVar varName)
                      return $ maybe "" id varValue
resolve (T t)    = do tmplName <- resolve t
                      body     <- asks (lookupTemplate tmplName)
                      return $ maybe "" show body

template1 :: Template
template1 = V (S "varName1")

template2 :: Template
template2 = T (S "varName2")

myEnv :: Environment
myEnv = Env {
      templates = [("varName1",template1),("varName2",template2)],
      variables = [("varName1","value1"),("varName2","value2")]
    }

main :: IO ()
main = do
    print $ runReader (resolve template1) myEnv
    print $ runReader (resolve template2) myEnv

main

In our last example let's implement now the code from the first chapter with the use of the **Reader monad**.

In [ ]:
processEnv :: Reader Environment String
processEnv = do
    conf1 <- func1
    conf2 <- func2
    conf3 <- func3
    conf4 <- func4
    let conf = conf1 ++ "\n\n" ++
               conf2 ++ "\n\n" ++
               conf3 ++ "\n\n" ++
               conf4 ++ "\n\n"
    return conf
  
func1 :: Reader Environment String
func1 = do
    env <- ask
    let content = "System path includes: \n" ++ param1 env
    return content

func2 :: Reader Environment String
func2 = do
    env <- ask
    let content = "Shell used is: \n" ++ param2 env
    return content

func3 :: Reader Environment String
func3 = do
    env <- ask
    let content = "Hostname is: \n" ++ param3 env
    return content

func4 :: Reader Environment String
func4 = do
    env <- ask
    let content = "Current home directory is: \n" ++ param4 env
    return content

main :: IO ()
main = do
    env <- loadEnv
    let configuration = runReader processEnv env
        filePath = param4 env ++ "/conf.txt"
    print "Writing configuration to file: conf.txt"
    writeFile filePath configuration

main

The `env` variable is passed from the `processEnv` function to all the other Reader monad functions.

Benefits compared to initial code: 
- you don't have to take care of passing around the environment data

- the Reader monads `func1` to `func4` have a cleaner type signature as the functions in the initial example.

## Recap

In this lesson we've discussed:

- the motivation for introducing the Reader monad type 

- the definition of the Reader monad type

- helper functions that work with the Reader monad

- examples that use the Reader monad type and its helper functions